# Generate ROMs from the Object Files in Tempest Source Disks

In [150]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

Get the source files from the 'historicalsource' github repository.

In [198]:
!rm -rf tempest
!git clone https://github.com/historicalsource/tempest

Cloning into 'tempest'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 45 (delta 8), reused 45 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 174.01 KiB | 1.89 MiB/s, done.
Resolving deltas: 100% (8/8), done.


The object files in the source dump are:

|File|Description
|---|---|
`TEMPST.LDA`| Object file for version 1 of Tempest
`ALEXEC.LDA` | Object file for version 2 of Tempest
`MBOX.SAV` | Object file for the MathBox Module
`STATE2.SAV` | Object file for the Vector Generator Module


While the `SAV` files are more or less raw binary output from the Atari MAC65 assembler, `LDA` is an PDP11/RT-11 file format. It's not a raw binary dump, instead the data is stored in chunks each one with with a header and checksum.

A chunk looks something like this (all words are little-endian):
```
Sentinel: 0x0100
Length: 0x0B00
Load Address: 0x0090
Data: 0x0102030405
Checksum: 0x09
```

You keep reading the chunks above until there are none left! See http://www.bitsavers.org/pdf/dec/pdp11/rt11/v5.6_Aug91/AA-PD6PA-TC_RT-11_Volume_and_File_Formats_Manual_Aug91.pdf. So let's start with `TEMPST.LDA` and extract the binary data from it.

## Generate ROMs for Tempest Version 1

A short function to parse LDA files.

In [164]:
word = lambda x: (x).to_bytes(2,"little")
read_word = lambda x: int.from_bytes(x,"little")

def parse_lda_file(filename):
    BLOCK_HEADER = b'\x01\x00'
    LDA_END = 0x0000
    
    al = open(filename,'rb')
    blocks = []
    while True:
        sentinel = al.read(2)
        if sentinel != BLOCK_HEADER:
            raise Exception("Block Header Expected But Got: ", sentinel)

        len_data = read_word(al.read(2)) - 6
        # Have we reached the end of our blocks?
        if len_data == LDA_END:
            break

        addr = read_word(al.read(2))
        data = al.read(len_data)
        checksum = al.read(1)
        blocks += [(hex(addr),addr,len(data),data)]
        
    # Use the addresses given in our datablocks to populate
    # a bytearray of all of the ROM data.
    output_bytes = bytearray(b'\x00' * 0xFFFF)
    for _,addr,len_data,data in blocks:
        output_bytes[addr:addr+len_data] = data

    return output_bytes

Let's use it to parse `TEMPST.LDA`.

In [156]:
output_bytes = parse_lda_file("tempest/TEMPST.LDA")

Now we have the 'raw' bytes for our ROMs in `output_bytes`.

The file `TEMPST.DOC` gives us some version control information for different builds of Tempest. This is what has for Version 1 of Tempest, it's at the very bottom of the file. The 'START ADDR' tells us where the different address ranges in our `output_bytes` should go and which ROM file they should be written to.

```
PROJECT NUMBER:	28903			PROJECT PROGRAMMER:	DAVE THEURER			PROJECT LEADER:	MORGAN HOFF

DOCUMENTATION DISK(S): 36A,36B,36C		PROJECT NAME:	TEMPEST			DATE:	8/26/81		Version: 1


Verification control filename: 002X1.DAT	Verification file: TEMPST.LDA


PART NUMBER	PCB LOCATION	DESCRIPTION	START ADDR.	RELEASE FILE NAME	GENERIC NAME	PART SIZE	BITS USED

136002-111	N3		VECT.GEN.ROM	3000		136002.111		8316E		2KX8		0-7
136002-112	R3		VECT.GEN.ROM	3800		136002.112		8316E		2KX8		0-7

[Cut the Program ROMs, which are a copy of the EPROMs below]

136002-123	N3		VECT.GEN.EPROM	3000		136002.123		2716		2KX8		0-7
136002-124	R3		VECT.GEN.EPROM	3800		136002.124		2716		2KX8		0-7

136002-113	D1		PROGRAM EPROM	9000		136002.113		2716		2KX8		0-7
136002-114	E1		PROGRAM EPROM	9800		136002.114		2716		2KX8		0-7
136002-115	F1		PROGRAM EPROM	A000		136002.115		2716		2KX8		0-7
136002-116	H1		PROGRAM EPROM	A800		136002.116		2716		2KX8		0-7
136002-117	J1		PROGRAM EPROM	B000		136002.117		2716		2KX8		0-7
136002-118	K1		PROGRAM EPROM	B800		136002.118		2716		2KX8		0-7
136002-119	L/M1		PROGRAM EPROM	C000		136002.119		2716		2KX8		0-7
136002-120	N1		PROGRAM EPROM	C800		136002.120		2716		2KX8		0-7
136002-121	P1		PROGRAM EPROM	D000		136002.121		2716		2KX8		0-7
136002-122	R1		PROGRAM EPROM	D800		136002.122		2716		2KX8		0-7

						    MASK
136002-125	D7		V.G.STATE ROM	1000(0F)*	STATE2.SAV		82S129		256X4		0-3

						    MASK
136002-132	L1(AUX)		MICRO. PROM	6800(F0)*	136002.1XX		82S129		256X4		4-7 *
136002-131	K1		MICRO. PROM	6800(0F)*	136002.1XX		82S129		256X4		0-3
136002-130	J1		MICRO. PROM	6900(F0)*	136002.1XX		82S129		256X4		4-7
136002-129	H1		MICRO. PROM	6900(0F)*	136002.1XX		82S129		256X4		0-3
136002-128	F1		MICRO. PROM	6A00(F0)*	136002.1XX		82S129		256X4		4-7
136002-127	E1		MICRO. PROM	6A00(0F)*	136002.1XX		82S129		256X4		0-3

136002.126	A1		MAPPING PROM	8400		136002.1XX		82S123		32X8		0-7
```

This more or less corresponds with the equivalent information in [MAME's manifest](https://github.com/mamedev/mame/blob/master/src/mame/atari/tempest.cpp) for version 1:

```cpp
ROM_START( tempest1 ) /* rev 1 */
	/* Roms are for Tempest Analog Vector-Generator PCB Assembly A037383-01 or A037383-02 */
	ROM_REGION( 0x10000, "maincpu", 0 )
	ROM_LOAD( "136002-113.d1",   0x9000, 0x0800, CRC(65d61fe7) SHA1(38a1e8a8f65b7887cf3e190269fe4ce2c6f818aa) )
	ROM_LOAD( "136002-114.e1",   0x9800, 0x0800, CRC(11077375) SHA1(ed8ff0ca969da6672a7683b93d4fcf2935a0d903) )
	ROM_LOAD( "136002-115.f1",   0xa000, 0x0800, CRC(f3e2827a) SHA1(bd04fcfbbba995e08c3144c1474fcddaaeb1c700) )
	ROM_LOAD( "136002-116.h1",   0xa800, 0x0800, CRC(7356896c) SHA1(a013ede292189a8f5a907de882ee1a573d784b3c) )
	ROM_LOAD( "136002-117.j1",   0xb000, 0x0800, CRC(55952119) SHA1(470d914fa52fce3786cb6330889876d3547dca65) )
	ROM_LOAD( "136002-118.k1",   0xb800, 0x0800, CRC(beb352ab) SHA1(f213166d3970e0bd0f29d8dea8d6afa6990cce38) )
	ROM_LOAD( "136002-119.lm1",  0xc000, 0x0800, CRC(a4de050f) SHA1(ea302e43a313a5a18115e74ddbaaedde0fbecda7) )
	ROM_LOAD( "136002-120.mn1",  0xc800, 0x0800, CRC(35619648) SHA1(48f1e8bed7ec6afa0b4c549a30e5ec331c071e40) )
	ROM_LOAD( "136002-121.p1",   0xd000, 0x0800, CRC(73d38e47) SHA1(9980606376a79ba94f8e2a325871a6c8d10d83fc) )
	ROM_LOAD( "136002-122.r1",   0xd800, 0x0800, CRC(796a9918) SHA1(c862a0d4ea330161e4c3cc8e5e9ad38893fffbd4) )
	ROM_RELOAD(                  0xf800, 0x0800 ) /* for reset/interrupt vectors */

	/* Vector ROM */
	ROM_REGION( 0x1000, "vectorrom", 0 )
	ROM_LOAD( "136002-123.np3",  0x0000, 0x0800, CRC(29f7e937) SHA1(686c8b9b8901262e743497cee7f2f7dd5cb3af7e) ) /* May be labeled "136002-111", same data */
	ROM_LOAD( "136002-124.r3",   0x0800, 0x0800, CRC(c16ec351) SHA1(a30a3662c740810c0f20e3712679606921b8ca06) ) /* May be labeled "136002-112", same data */

	/* AVG PROM */
	ROM_REGION( 0x100, "avg:prom", 0 )
	ROM_LOAD( "136002-125.d7",   0x0000, 0x0100, CRC(5903af03) SHA1(24bc0366f394ad0ec486919212e38be0f08d0239) )

	/* Mathbox PROMs */
	ROM_REGION( 0x20, "user2", 0 )
	ROM_LOAD( "136002-126.a1",   0x0000, 0x0020, CRC(8b04f921) SHA1(317b3397482f13b2d1bc21f296d3b3f9a118787b) )

	ROM_REGION32_BE( 0x400, "user3", 0 )
	ROMX_LOAD( "136002-132.l1", 0, 0x100, CRC(2af82e87) SHA1(3816835a9ccf99a76d246adf204989d9261bb065), ROM_NIBBLE | ROM_SHIFT_NIBBLE_LO | ROM_SKIP(3))
	ROMX_LOAD( "136002-131.k1", 0, 0x100, CRC(b31f6e24) SHA1(ce5f8ca34d06a5cfa0076b47400e61e0130ffe74), ROM_NIBBLE | ROM_SHIFT_NIBBLE_HI | ROM_SKIP(3))
	ROMX_LOAD( "136002-130.j1", 1, 0x100, CRC(8119b847) SHA1(c4fbaedd4ce1ad6a4128cbe902b297743edb606a), ROM_NIBBLE | ROM_SHIFT_NIBBLE_LO | ROM_SKIP(3))
	ROMX_LOAD( "136002-129.h1", 1, 0x100, CRC(09f5a4d5) SHA1(d6f2ac07ca9ee385c08831098b0dcaf56808993b), ROM_NIBBLE | ROM_SHIFT_NIBBLE_HI | ROM_SKIP(3))
	ROMX_LOAD( "136002-128.f1", 2, 0x100, CRC(823b61ae) SHA1(d99a839874b45f64e14dae92a036e47a53705d16), ROM_NIBBLE | ROM_SHIFT_NIBBLE_LO | ROM_SKIP(3))
	ROMX_LOAD( "136002-127.e1", 2, 0x100, CRC(276eadd5) SHA1(55718cd8ec4bcf75076d5ef0ee1ed2551e19d9ba), ROM_NIBBLE | ROM_SHIFT_NIBBLE_HI | ROM_SKIP(3))
ROM_END
```

So using the information above we can generate rom files using the content of the `output_bytes` array we constructed from parsing the data blocks in `TEMPST.LDA`. All we need to do is use the addresses and lengths given in the Tempest documentation:

In [157]:
OUT_DIR = "TEMPST.LDA"
open(f"{OUT_DIR}/tempest/136002-111.d1",'wb').write(bytes(output_bytes[0x3000:0x3800]))
open(f"{OUT_DIR}/tempest/136002-123.np3",'wb').write(bytes(output_bytes[0x3000:0x3800]))
open(f"{OUT_DIR}/tempest/136002-112.d1",'wb').write(bytes(output_bytes[0x3800:0x4000]))
open(f"{OUT_DIR}/tempest/136002-124.r3",'wb').write(bytes(output_bytes[0x3800:0x4000]))

open(f"{OUT_DIR}/tempest/136002-113.d1",'wb').write(bytes(output_bytes[0x9000:0x9800]))
open(f"{OUT_DIR}/tempest/136002-114.e1",'wb').write(bytes(output_bytes[0x9800:0xA000]))
open(f"{OUT_DIR}/tempest/136002-115.f1",'wb').write(bytes(output_bytes[0xA000:0xA800]))
open(f"{OUT_DIR}/tempest/136002-116.h1",'wb').write(bytes(output_bytes[0xA800:0xB000]))
open(f"{OUT_DIR}/tempest/136002-117.j1",'wb').write(bytes(output_bytes[0xB000:0xB800]))
open(f"{OUT_DIR}/tempest/136002-118.k1",'wb').write(bytes(output_bytes[0xB800:0xC000]))
open(f"{OUT_DIR}/tempest/136002-119.lm1",'wb').write(bytes(output_bytes[0xC000:0xC800]))
open(f"{OUT_DIR}/tempest/136002-120.mn1",'wb').write(bytes(output_bytes[0xC800:0xD000]))
open(f"{OUT_DIR}/tempest/136002-121.p1",'wb').write(bytes(output_bytes[0xD000:0xD800]))
open(f"{OUT_DIR}/tempest/136002-122.r1",'wb').write(bytes(output_bytes[0xD800:0xE000]))

2048

We do the same with `MBOX.SAV`. We don't need to do any parsing of this file. We can just extract the bytes more or less directly from it and write to the appropriate ROM File.

In [158]:
mathbox_bytes = open("tempest/MBOX.SAV",'rb').read()
open(f"{OUT_DIR}/tempest/136002-126.a1",'wb').write(mathbox_bytes[0x8400:0x8420])
# 136002-127.e1 contains the low 4 bits of each byte in 0x6A00:0x6B00 in MBOX.SAV
open(f"{OUT_DIR}/tempest/136002-127.e1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6A00:0x6B00]]))
# 136002-128.f1 contains the high 4 bits (right-shifted) of each byte in 0x6A00:0x6B00 in MBOX.SAV
open(f"{OUT_DIR}/tempest/136002-128.f1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6A00:0x6B00]]))

# 136002-129.h1 contains the low 4 bits of each byte in 0x6900:0x6A00 in MBOX.SAV
open(f"{OUT_DIR}/tempest/136002-129.h1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6900:0x6A00]]))
# 136002-130.j1 contains the high 4 bits (right-shifted) of each byte in 0x6900:0x6A00 in MBOX.SAV
open(f"{OUT_DIR}/tempest/136002-130.j1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6900:0x6A00]]))

# 136002-131.k1 contains the low 4 bits of each byte in 0x6800:0x6900 in MBOX.SAV
open(f"{OUT_DIR}/tempest/136002-131.k1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6800:0x6900]]))
# 136002-132.l1 contains the high 4 bits (right-shifted) of each byte in 0x6800:0x6900 in MBOX.SAV
open(f"{OUT_DIR}/tempest/136002-132.l1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6800:0x6900]]))


256

And finally we extract the relevant bytes from `STATE2.SAV`:

In [159]:
vecgen_bytes = open("tempest/STATE2.SAV",'rb').read()
open(f"{OUT_DIR}/tempest/136002-125.d7",'wb').write(vecgen_bytes[0x1000:0x1100])

256

Now we can run `mame` with the ROMS we've generated. If you haven't already you will need to install mame like so:
```
sudo apt install mame
```

In [162]:
!mame -window -rompath ./TEMPST.LDA/ tempest1

Unable to find the BGFX path bgfx, please install it or fix the bgfx_path setting to use the BGFX renderer.


It works. And it doesn't give us any checksum warnings, which means the ROMs we generated are byte for byte identical to the ones expected of the first version of Tempest.

## Generate ROMs for Tempest Version 2A (Alt)

The other object file in the source dump is `ALEXEC.LDA`.

In [165]:
output_bytes = parse_lda_file("tempest/ALEXEC.LDA")

The manifest for Version 2A (alt) in `TEMPST.DOC` tells us what ROM names to use. 


```
Project #:  28903			Project Programmer:  Dave Theurer		Project Leader:  Morgan Hoff

Documentation Disk(s): 36A,36B,36C		Project Name:  Tempest			Date:  12-17-81		Version: 2A (alt)


Verification control filename: 002X2.DAT	Verification file: ALEXEC.LDA


Part #		PCB Loc.	Description		Start Addr	Generic name	Part Size	Bits Used
------		--------	-----------		----------	------------	---------	---------	

136002-138	N/P3		VG EROM			3000		2532		4KX8		7->0				

136002-237	P1		PROGRAM EROM		9000		2532		4KX8		7->0
136002-136	L/M1		PROGRAM EROM		A000		2532		4KX8		7->0
136002-235	J1		PROGRAM EROM		B000		2532		4KX8		7->0
136002-134	F1		PROGRAM EROM		C000		2532		4KX8		7->0
136002-133	D1		PROGRAM EROM		D000		2532		4KX8		7->0
				(ind. self test)


General Information:
--------------------

Assembler used:  MAC65

Main program source files:
	Includes:  ALCOMN, HLL65, COIN65, VGMC, ANVGAN
	Files:  ALWELG,ALSCO2,ALDIS2,ALEXEC,ALSOUN,ALVROM,ALCOIN,ALLANG,ALHAR2,ALTES2,ALEARO,ALVGUT

Picture file:

Linker used:  LINKM

Linker command:  ALEXEC/L,ALEXEC/A=ALWELG,ALSCO2,ALDIS2,ALEXEC,ALSOUN,ALVROM,ALCOIN,ALLANG,ALHAR2,ALTES2,ALEARO,ALVGUT
```


The corresponding entry in Mame's manifest given as 'Rev 3'. It is the 'last' release of Tempest so is given simply as `tempest`. Notice that the addresses for the ROMs are reversed in the Mame entry. `136002-133.d1` is loaded to `0x9000` rather than `0xD000` as specified by the Atari version control doc above.

```cpp
ROM_START( tempest ) /* rev 3 */
	/* Roms are for Tempest Analog Vector-Generator PCB Assembly A037383-03 or A037383-04 */
	ROM_REGION( 0x10000, "maincpu", 0 )
	ROM_LOAD( "136002-133.d1",  0x9000, 0x1000, CRC(1d0cc503) SHA1(7bef95db9b1102d6b1166bda0ccb276ef4cc3764) ) /* 136002-113 + 136002-114 */
	ROM_LOAD( "136002-134.f1",  0xa000, 0x1000, CRC(c88e3524) SHA1(89144baf1efc703b2336774793ce345b37829ee7) ) /* 136002-115 + 136002-316 */
	ROM_LOAD( "136002-235.j1",  0xb000, 0x1000, CRC(a4b2ce3f) SHA1(a5f5fb630a48c5d25346f90d4c13aaa98f60b228) ) /* 136002-217 + 136002-118 */
	ROM_LOAD( "136002-136.lm1", 0xc000, 0x1000, CRC(65a9a9f9) SHA1(73aa7d6f4e7093ccb2d97f6344f354872bcfd72a) ) /* 136002-119 + 136002-120 */
	ROM_LOAD( "136002-237.p1",  0xd000, 0x1000, CRC(de4e9e34) SHA1(04be074e45bf5cd95a852af97cd04e35b7f27fc4) ) /* 136002-121 + 136002-222 */
	ROM_RELOAD(                 0xf000, 0x1000 ) /* for reset/interrupt vectors */

	/* Vector ROM */
	ROM_REGION( 0x1000, "vectorrom", 0 )
	ROM_LOAD( "136002-138.np3", 0x0000, 0x1000, CRC(9995256d) SHA1(2b725ee1a57d423c7d7377a1744f48412e0f2f69) )

	/* AVG PROM */
	ROM_REGION( 0x100, "avg:prom", 0 )
	ROM_LOAD( "136002-125.d7",   0x0000, 0x0100, CRC(5903af03) SHA1(24bc0366f394ad0ec486919212e38be0f08d0239) )

	/* Mathbox PROMs */
	ROM_REGION( 0x20, "user2", 0 )
	ROM_LOAD( "136002-126.a1",   0x0000, 0x0020, CRC(8b04f921) SHA1(317b3397482f13b2d1bc21f296d3b3f9a118787b) )

	ROM_REGION32_BE( 0x400, "user3", 0 )
	ROMX_LOAD( "136002-132.l1", 0, 0x100, CRC(2af82e87) SHA1(3816835a9ccf99a76d246adf204989d9261bb065), ROM_NIBBLE | ROM_SHIFT_NIBBLE_LO | ROM_SKIP(3))
	ROMX_LOAD( "136002-131.k1", 0, 0x100, CRC(b31f6e24) SHA1(ce5f8ca34d06a5cfa0076b47400e61e0130ffe74), ROM_NIBBLE | ROM_SHIFT_NIBBLE_HI | ROM_SKIP(3))
	ROMX_LOAD( "136002-130.j1", 1, 0x100, CRC(8119b847) SHA1(c4fbaedd4ce1ad6a4128cbe902b297743edb606a), ROM_NIBBLE | ROM_SHIFT_NIBBLE_LO | ROM_SKIP(3))
	ROMX_LOAD( "136002-129.h1", 1, 0x100, CRC(09f5a4d5) SHA1(d6f2ac07ca9ee385c08831098b0dcaf56808993b), ROM_NIBBLE | ROM_SHIFT_NIBBLE_HI | ROM_SKIP(3))
	ROMX_LOAD( "136002-128.f1", 2, 0x100, CRC(823b61ae) SHA1(d99a839874b45f64e14dae92a036e47a53705d16), ROM_NIBBLE | ROM_SHIFT_NIBBLE_LO | ROM_SKIP(3))
	ROMX_LOAD( "136002-127.e1", 2, 0x100, CRC(276eadd5) SHA1(55718cd8ec4bcf75076d5ef0ee1ed2551e19d9ba), ROM_NIBBLE | ROM_SHIFT_NIBBLE_HI | ROM_SKIP(3))
ROM_END
```

In [145]:
OUT_DIR="ALEXEC.LDA/tempest/"

open(f"{OUT_DIR}136002-123.np3",'wb').write(bytes(output_bytes[0x3000:0x3800]))
open(f"{OUT_DIR}136002-124.r3",'wb').write(bytes(output_bytes[0x3800:0x4000]))

open(f"{OUT_DIR}136002-138.np3",'wb').write(bytes(output_bytes[0x3000:0x4000]))

# Notice we have the addresses in reverse order, e.g. 136002-237 is written to 0xD000 rather
# than 0x9000.
open(f"{OUT_DIR}136002-237.p1",'wb').write(bytes(output_bytes[0xD000:0xE000]))
open(f"{OUT_DIR}136002-136.lm1",'wb').write(bytes(output_bytes[0xC000:0xD000]))
open(f"{OUT_DIR}136002-235.j1",'wb').write(bytes(output_bytes[0xB000:0xC000]))
open(f"{OUT_DIR}136002-134.f1",'wb').write(bytes(output_bytes[0xA000:0xB000]))
open(f"{OUT_DIR}136002-133.d1",'wb').write(bytes(output_bytes[0x9000:0xA000]))


4096

As with version 1 we need the mathbox and vector generator ROMs. They're unchanged, so we just copy them in again.

In [146]:
mathbox_bytes = open("tempest/MBOX.SAV",'rb').read()
open(f"{OUT_DIR}/136002-126.a1",'wb').write(mathbox_bytes[0x8400:0x8420])
# 136002-127.e1 contains the low 4 bits of each byte in 0x6A00:0x6B00 in MBOX.SAV
open(f"{OUT_DIR}/136002-127.e1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6A00:0x6B00]]))
# 136002-128.f1 contains the high 4 bits (right-shifted) of each byte in 0x6A00:0x6B00 in MBOX.SAV
open(f"{OUT_DIR}/136002-128.f1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6A00:0x6B00]]))

# 136002-129.h1 contains the low 4 bits of each byte in 0x6900:0x6A00 in MBOX.SAV
open(f"{OUT_DIR}/136002-129.h1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6900:0x6A00]]))
# 136002-130.j1 contains the high 4 bits (right-shifted) of each byte in 0x6900:0x6A00 in MBOX.SAV
open(f"{OUT_DIR}/136002-130.j1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6900:0x6A00]]))

# 136002-131.k1 contains the low 4 bits of each byte in 0x6800:0x6900 in MBOX.SAV
open(f"{OUT_DIR}/136002-131.k1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6800:0x6900]]))
# 136002-132.l1 contains the high 4 bits (right-shifted) of each byte in 0x6800:0x6900 in MBOX.SAV
open(f"{OUT_DIR}/136002-132.l1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6800:0x6900]]))


256

In [147]:
vecgen_bytes = open("tempest/STATE2.SAV",'rb').read()
open(f"{OUT_DIR}/136002-125.d7",'wb').write(vecgen_bytes[0x1000:0x1100])

256

Now let's see if it works:

In [163]:
!mame -window -rompath ./ALEXEC.LDA/ tempest

Unable to find the BGFX path bgfx, please install it or fix the bgfx_path setting to use the BGFX renderer.
Average speed: 99.99% (2 seconds)


It works - and we don't get any checksum errors. So `ALEXEC.LDA` contains the binary data for 'Revision 3', but is given as 2A(Alt) in Tempest's project documentation.

## But, what are the differences?

```
Project #: 28903	Project Programmer: Dave Theurer	Project Leader: Morgan Hoff

Documentation Disk(s): 36A,36B,36C	Project Name: Tempest	Date: 11-25-81	Version: 2


Verification control filename: 002X2.DAT	Verification file: ALEXEC.LDA


Part #	PCB Loc.	Description		Start Addr	Generic name	Part Size	Bits Used
------	--------	-----------		----------	------------	---------	---------

136002-123	N3	Vect.Gen.Erom		3000		2716		2Kx8		0-7
136002-124	R3	Vect.Gen.Erom		3800		2716		2Kx8		0-7

136002-113	D1	Program Erom		9000		2716		2Kx8		0-7
136002-114	E1	Program Erom		9800		2716		2Kx8		0-7
136002-115	F1	Program Erom		A000		2716		2Kx8		0-7
136002-216	H1	Program Erom		A800		2716		2Kx8		0-7
136002-217	J1	Program Erom		B000		2716		2Kx8		0-7
136002-118	K1	Program Erom		B800		2716		2Kx8		0-7
136002-119	L/M1	Program Erom		C000		2716		2Kx8		0-7
136002-120	N1	Program Erom		C800		2716		2Kx8		0-7
136002-121	P1	Program Erom		D000		2716		2Kx8		0-7
136002-222	R1	Program Erom		D800		2716		2Kx8		0-7

General Information:

Assembler used: MAC65

Main program source files:
	Includes: ALCOMN, HLL65, COIN65, VGMC, ANVGAN
	Files: ALWELG,ALSCO2,ALDIS2,ALEXEC,ALSOUN,ALVROM,ALCOIN,ALLANG,ALHAR2,ALTES2,ALEARO,ALVGUT
```

In [168]:
OUT_DIR="ALEXEC.LDA/tempest2/"
open(f"{OUT_DIR}136002-123.np3",'wb').write(bytes(output_bytes[0x3000:0x3800]))
open(f"{OUT_DIR}136002-124.r3",'wb').write(bytes(output_bytes[0x3800:0x4000]))

open(f"{OUT_DIR}136002-113.d1",'wb').write(bytes(output_bytes[0x9000:0x9800]))
open(f"{OUT_DIR}136002-114.e1",'wb').write(bytes(output_bytes[0x9800:0xA000]))
open(f"{OUT_DIR}136002-115.f1",'wb').write(bytes(output_bytes[0xA000:0xA800]))

open(f"{OUT_DIR}136002-216.h1",'wb').write(bytes(output_bytes[0xA800:0xB000]))
open(f"{OUT_DIR}136002-116.h1",'wb').write(bytes(output_bytes[0xA800:0xB000]))

open(f"{OUT_DIR}136002-217.j1",'wb').write(bytes(output_bytes[0xB000:0xB800]))
open(f"{OUT_DIR}136002-118.k1",'wb').write(bytes(output_bytes[0xB800:0xC000]))
open(f"{OUT_DIR}136002-119.lm1",'wb').write(bytes(output_bytes[0xC000:0xC800]))
open(f"{OUT_DIR}136002-120.mn1",'wb').write(bytes(output_bytes[0xC800:0xD000]))
open(f"{OUT_DIR}136002-121.p1",'wb').write(bytes(output_bytes[0xD000:0xD800]))
open(f"{OUT_DIR}136002-222.r1",'wb').write(bytes(output_bytes[0xD800:0xE000]))


2048

In [169]:
!mame -window -rompath /home/robert/Dev/tempest/notebooks/ALEXEC.LDA tempest2

Unable to find the BGFX path bgfx, please install it or fix the bgfx_path setting to use the BGFX renderer.
136002-116.h1 WRONG CHECKSUMS:
    EXPECTED: CRC(7356896c) SHA1(a013ede292189a8f5a907de882ee1a573d784b3c)
       FOUND: CRC(aeb0f7e9) SHA1(a5cc25015b98692673cfc1c7c2e9634efd750870)


In [170]:
!diff -y -W 150 <(dd if=ALEXEC.LDA/tempest2/136002-216.h1|xxd) <(dd if=roms/136002-116.h1|xxd) |grep '  |	'

4+0 records in
4+0 records out
4+0 records in
4+0 records out
2048 bytes (2.0 kB, 2.0 KiB) copied, 2.2166e-05 s, 92.4 MB/s
2048 bytes (2.0 kB, 2.0 KiB) copied, 1.9622e-05 s, 104 MB/s
000000a0: 8d25 0160 b98a 0229 fc99 8a02 4c98 a3e1  .%.`...)....L...	  |	000000a0: 8d25 0160 b98a 0229 fc99 8a02 4c98 a3b2  .%.`...)....L...
00000260: 69aa a930 a200 2017 ab20 92aa 4ce7 a820  i..0.. .. ..L.. 	  |	00000260: 69aa a930 a200 2017 ab20 92aa 4cb4 a820  i..0.. .. ..L.. 


To understand this difference we'll have to do some digging into the sources. We will need to [build the sources](../atari_build/README.md) and then [extract the object files we get into a folder where we can get at them](./Extract%20Object%20Files%20from%20Tempest%20RK05%20Disk%20after%20Assembling%20and%20Linking.ipynb).

In [191]:
!diff -y -W 150 ../src/ALDISP.MAC ../src/ALDIS2.MAC |grep '  |	'

In [194]:
!diff -c2 ../src/ALDISP.MAC ../src/ALDIS2.MAC

*** ../src/ALDISP.MAC	2024-10-19 10:38:34.192143691 +0100
--- ../src/ALDIS2.MAC	2024-10-19 10:38:34.196143907 +0100
***************
*** 79,83 ****





--- 79,83 ----





***************
*** 337,341 ****


! 


--- 337,341 ----





***************
*** 2847,2851 ****


! ;INPUT:ACC=LEVEL #-1


--- 2847,2851 ----





***************
*** 3282,3284 ****



\ No newline at end of file
--- 3282,3283 ----


In [195]:
!diff -c2 ../src/ALHARD.MAC ../src/ALHAR2.MAC

*** ../src/ALHARD.MAC	2024-10-19 10:38:34.192143691 +0100
--- ../src/ALHAR2.MAC	2024-10-19 10:38:34.192143691 +0100
***************
*** 20,30 ****











--- 20,30 ----













In [196]:
!diff -c2 ../src/ALSCOR.MAC ../src/ALSCO2.MAC

*** ../src/ALSCOR.MAC	2024-10-19 10:38:34.196143907 +0100
--- ../src/ALSCO2.MAC	2024-10-19 10:38:34.192143691 +0100
***************
*** 72,76 ****





--- 72,76 ----





***************
*** 313,317 ****





--- 313,317 ----







In [197]:
!diff -c2 ../src/ALTEST.MAC ../src/ALTES2.MAC

*** ../src/ALTEST.MAC	2024-10-19 10:38:34.196143907 +0100
--- ../src/ALTES2.MAC	2024-10-19 10:38:34.196143907 +0100
***************
*** 19,23 ****





--- 19,23 ----





***************
*** 244,249 ****






--- 244,249 ----






***************
*** 263,267 ****





--- 263,267 ----





***************
*** 288,295 ****
--- 288,299 ----












***************
*** 400,404 ****





--- 404,408 ----





***************
*** 440,443 ****
--- 444,450 ----







***************
*** 495,499 ****





--- 502,506 ----





***************
*** 562,635 ****










































































--- 569,572 ----
***************
*** 639,654 ****
















--- 576,599 ----
























***************
*** 672,692 ****





















--- 617,707 ----



























































































***************
*** 724,728 ****





--- 739,743 ----





***************
*** 731,735 ***